<h1> Q1: Linear Regression </h1>

In [3]:
import os
import autograd.numpy as np  # when testing gradient
from cvxopt import matrix, solvers
import pandas as pd


In [5]:
pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 10.2 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
#Q1a)

def minimizeL2(X,y):
    #from lecture, we know the analytic solution to L2 loss
    # is   w = ((XtX)^-1)Xty

    x_trans = X.T

    #(XtX)^-1
    firstpart = np.linalg.inv(x_trans @ X)
    #secondpart XtY
    secpart = np.dot(x_trans, y)

    return (firstpart @ secpart)


# Sample data for testing
X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([7, 8, 9])    
    
    
# Test the function
w = minimizeL2(X, y)
print("Weights w:", w)

# Check if the result makes sense by calculating X @ w and comparing with y
y_pred = X @ w
print("Predicted y:", y_pred)
print("Original y:", y)


Weights w: [-6.   6.5]
Predicted y: [7. 8. 9.]
Original y: [7 8 9]


In [25]:
#Q1b)

def minimizeL1(X,y):

    n, d = X.shape
    Identity = np.eye(n)
    nOnes = np.ones(n)
    objfunc = np.concatenate([np.zeros(d), nOnes])  # [0's for w, 1's for delta]

    #constraints
    con1 = np.hstack([X, -Identity])  #Xw - y <= delta
    con2 = np.hstack([-X, -Identity])  #y - Xw <= delta
    constraints = np.vstack([con1, con2]) #combine to make the constraint array

    #y portion of |Xw-y| < delta
    h1 = y   #Xw - delta <= y
    h2 = -y   #y-Xw - delta <= -y
    h = np.concatenate([h1,h2])
    h = h.reshape(-1, 1) #turn to col vector
    h = matrix(h, tc='d')  # Ensure h is a cvxopt matrix in double precision

    constraints_cvxopt = matrix(constraints)
    h_cvxopt = matrix(h)
    objfunc_cvxopt = matrix(objfunc)

    solvers.options['show_progress'] = False  # Silence solver output
    solution = solvers.lp(objfunc_cvxopt, constraints_cvxopt, h_cvxopt)

    return np.array(solution['x'])[:d]


# Testing the function
X = np.array([[1, 2], [2, 3], [3, 4]])  # 3 samples, 2 features
y = np.array([1, 2, 3])  # Target values

w = minimizeL1(X, y)

print("Computed weights (w) \n:", w)

y_pred = X @ w 
print("Predicted y: \n", y_pred)
print("Original y: \n", y)

l1_loss = np.sum(np.abs(y - y_pred))
print("L1 loss:", l1_loss)


Computed weights (w) 
: [[1.00000000e+00]
 [2.56881886e-16]]
Predicted y: 
 [[1.]
 [2.]
 [3.]]
Original y: 
 [1 2 3]
L1 loss: 8.0


In [26]:
#Q1c)

def minimizeLinf(X, y):
    
    n, d = X.shape  # n: number of data points, d: number of features
    objfunc = np.concatenate([np.zeros(d), [1]])  # Objective: [0's for w, 1 for delta]
    Identity = np.eye(n)

    
    con1 = np.hstack([X, -np.ones((n, 1))])   # Xw - delta <= y
    con2 = np.hstack([-X, -np.ones((n, 1))])  # y - Xw <= delta
    constraints = np.vstack([con1, con2])

    # y portion of |Xw - y| <= delta
    h1 = y  # Xw - y <= delta)
    h2 = -y  # y - Xw <= delta)
    h = np.concatenate([h1, h2])

    constraints_cvxopt = matrix(constraints, tc='d')
    h_cvxopt = matrix(h, tc='d')
    objfunc_cvxopt = matrix(objfunc, tc='d')

    solvers.options['show_progress'] = False
    solution = solvers.lp(objfunc_cvxopt, constraints_cvxopt, h_cvxopt)

    # Return the optimized weights (w), which are the first d elements of the solution
    return np.array(solution['x'])[:d]


# Testing the function
X = np.array([[1, 2], [2, 3], [3, 4]])  # 3 samples, 2 features
y = np.array([1, 2, 3])  # Target values

w = minimizeLinf(X, y)

print("Computed weights (w) \n:", w)

y_pred = X @ w 
print("Predicted y: \n", y_pred)
print("Original y: \n", y)

l1_loss = np.sum(np.abs(y - y_pred))
print("Linf loss:", l1_loss)

Computed weights (w) 
: [[1.00000000e+00]
 [5.89705064e-17]]
Predicted y: 
 [[1.]
 [2.]
 [3.]]
Original y: 
 [1 2 3]
Linf loss: 8.0


In [29]:
#d

def l2_Loss(w, X,y):
    print(f"Shape of X: {X.shape}")
    print(f"Shape of w: {w.shape}")

    predicted_diff = (np.linalg.norm(X @ w - y)**2)
    relative_result = predicted_diff / (2*X.shape[0])
    return relative_result


def l1_Loss(w, X,y):
    predicted = np.abs(X@w - y)
    avg = np.mean(predicted)
    return avg

def lInf_Loss(w, X,y):
    diff = np.abs(X@w-y)
    max_val = np.amax(diff)
    return max_val


def compute_loss(w_l2, w_l1,w_Linf,X,y):
    results = np.zeros([3, 3])

    results[0,0] = l2_Loss(w_l2, X,y)
    results[1,0] = l2_Loss(w_l1,X,y)
    results[2,0] = l2_Loss(w_Linf, X, y)
    
    results[0,1] = l1_Loss(w_l2, X,y)
    results[1,1] = l1_Loss(w_l1,X,y)
    results[2,1] = l1_Loss(w_Linf, X, y)

    results[0,2] = lInf_Loss(w_l2, X,y)
    results[1,2] = lInf_Loss(w_l1,X,y)
    results[2,2] = lInf_Loss(w_Linf, X, y)

    return results

    

def synRegExperiments():
    def genData(n_points): 
        X = np.random.randn(n_points, d) # input matrix
        X = np.concatenate((np.ones((n_points, 1)), X), axis=1) # augment input
        y = X @ w_true + np.random.randn(n_points, 1) * noise # ground truth label 
        return X, y
    
    n_runs = 100 
    n_train = 30 
    n_test = 1000
    d=5
    noise = 0.2
    train_loss = np.zeros([n_runs, 3, 3]) # n_runs * n_models * n_metrics 
    test_loss = np.zeros([n_runs, 3, 3]) # n_runs * n_models * n_metrics

    for r in range(n_runs):
        w_true = np.random.randn(d + 1, 1)
        Xtrain, ytrain = genData(n_train)
        Xtest, ytest = genData(n_test)
        # Learn different models from the training data
        w_L2 = minimizeL2(Xtrain, ytrain)
        w_L1 = minimizeL1(Xtrain, ytrain)
        w_Linf = minimizeLinf(Xtrain, ytrain)

        # TODO: Evaluate the three models' performance (for each model,
        #       calculate the L2, L1 and L infinity losses on the training
        #       data). Save them to `train_loss`
        train_loss[r] = compute_loss(w_L2, w_L1,w_Linf,Xtrain,ytrain)

        # TODO: Evaluate the three models' performance (for each model,
            #       calculate the L2, L1 and L infinity losses on the test
            #       data). Save them to `test_loss`
        test_loss[r] = compute_loss(w_L2, w_L1,w_L1,Xtest,ytest)

       # Compute the average losses over runs
    avg_train_loss = np.mean(train_loss, axis=0)
    avg_test_loss = np.mean(test_loss, axis=0)

    # Print the results
    print("Average Training Loss (3x3 Matrix):")
    print(avg_train_loss)
    print("\nAverage Test Loss (3x3 Matrix):")
    print(avg_test_loss)

    # Return the average losses
    return avg_train_loss, avg_test_loss

synRegExperiments()



Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (1000, 6)
Shape of w: (6, 1)
Shape of X: (30, 6)
Shape of w: (6, 1)
S

(array([[0.01557454, 0.14042051, 0.41440518],
        [0.01758925, 0.13215385, 0.48218991],
        [0.02112436, 0.17405729, 0.30827302]]),
 array([[0.02487018, 0.17757422, 0.77207275],
        [0.02754015, 0.18644787, 0.80998377],
        [0.02754015, 0.18644787, 0.80998377]]))